In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import requests
import sqlite3
import os

In [2]:
def download(url, filename):
  response = requests.get(url)
  response.raise_for_status()  # Raise an exception for bad status codes
  with open(filename, "wb") as f:
    f.write(response.content)

In [3]:
census_data = pd.read_csv('/content/drive/MyDrive/Datasets/ChicagoCensusData.csv')
census_data.head()

,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_OF_HOUSING_CROWDED,PERCENT_HOUSEHOLDS_BELOW_POVERTY,PERCENT_AGED_16__UNEMPLOYED,PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA,PERCENT_AGED_UNDER_18_OR_OVER_64,PER_CAPITA_INCOME,HARDSHIP_INDEX
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0


In [4]:
public_schools = pd.read_csv('/content/drive/MyDrive/Datasets/ChicagoPublicSchools.csv')
public_schools.head()

,School_ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School",Street_Address,City,State,ZIP_Code,Phone_Number,Link,Network_Manager,...,Freshman_on_Track_Rate__,X_COORDINATE,Y_COORDINATE,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,Ward,Police_District,Location
0,610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressRep...,Fullerton Elementary Network,...,NDA,1171699.458,1915829.428,41.924497,-87.644522,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"
1,610281,Adam Clayton Powell Paideia Community Academy ...,ES,7511 S South Shore Dr,Chicago,IL,60649,(773) 535-6650,http://schoolreports.cps.edu/SchoolProgressRep...,Skyway Elementary Network,...,NDA,1196129.985,1856209.466,41.760324,-87.556736,43,SOUTH SHORE,7,4,"(41.76032435, -87.55673627)"
2,610185,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago,IL,60652,(773) 535-2280,http://schoolreports.cps.edu/SchoolProgressRep...,Midway Elementary Network,...,NDA,1148427.165,1851012.215,41.747111,-87.731702,70,ASHBURN,13,8,"(41.74711093, -87.73170248)"
3,609993,Agustin Lara Elementary Academy,ES,4619 S Wolcott Ave,Chicago,IL,60609,(773) 535-4389,http://schoolreports.cps.edu/SchoolProgressRep...,Pershing Elementary Network,...,NDA,1164504.290,1873959.199,41.809757,-87.672145,61,NEW CITY,20,9,"(41.8097569, -87.6721446)"
4,610513,Air Force Academy High School,HS,3630 S Wells St,Chicago,IL,60609,(773) 535-1590,http://schoolreports.cps.edu/SchoolProgressRep...,Southwest Side High School Network,...,91.8,1175177.622,1880745.126,41.828146,-87.632794,34,ARMOUR SQUARE,11,9,"(41.82814609, -87.63279369)"


In [5]:
crime_data = pd.read_csv('/content/drive/MyDrive/Datasets/ChicagoCrimeData.csv')
crime_data.head()

,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,...,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
0,3512276,HK587712,2004-08-28,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,False,False,...,9,14.0,58.0,6,1155838.0,1873050.0,2004,41.807440,-87.703956,"(41.8074405, -87.703955849)"
1,3406613,HK456306,2004-06-26,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,False,False,...,11,27.0,23.0,6,1152206.0,1906127.0,2004,41.898280,-87.716406,"(41.898279962, -87.716405505)"
2,8002131,HT233595,2011-04-04,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,False,False,...,2,3.0,38.0,6,1177436.0,1876313.0,2011,41.815933,-87.624642,"(41.815933131, -87.624642127)"
3,7903289,HT133522,2010-12-30,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,...,4,7.0,46.0,6,1194622.0,1850125.0,2010,41.743665,-87.562463,"(41.743665322, -87.562462756)"
4,10402076,HZ138551,2016-02-02,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,False,False,...,8,15.0,66.0,6,1155240.0,1860661.0,2016,41.773455,-87.706480,"(41.773455295, -87.706480471)"


In [7]:
# Establish a connection to the SQLite database
conn = sqlite3.connect('FinalDB.db')

# Verify the connection (optional)
print("Database connection established.")

Database connection established.


In [8]:
# Load the data into the database tables

census_data.to_sql('CENSUS_DATA', conn, if_exists='replace', index=False)
public_schools.to_sql('PUBLIC_SCHOOLS', conn, if_exists='replace', index=False)
crime_data.to_sql('CRIME_DATA', conn, if_exists='replace', index=False)

print("Data has been loaded into the database tables.")

Data has been loaded into the database tables.


In [9]:
# Close the connection
conn.close()

In [10]:
# Loading the SQL magic module

# Define the URLs for the datasets (replace with actual URLs)
census_data_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01'
public_schools_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01'
crime_data_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01'

In [11]:
# Establish a connection between SQL magic module and the database FinalDB.db

# Step 1: Install the ipython-sql package
!pip install -q ipython-sql

# Step 2: Load the SQL magic extension
%load_ext sql

# Step 3: Establish a connection to the SQLite database
# Use the appropriate SQLite URI format
%sql sqlite:///FinalDB.db

# You can now run SQL commands using the SQL magic
# For example, you can check the tables in the database
%sql SELECT name FROM sqlite_master WHERE type='table';

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.3 MB/s eta 0:00:00
 * sqlite:///FinalDB.db
Done.


name
CENSUS_DATA
PUBLIC_SCHOOLS
CRIME_DATA


In [13]:
# Problem 1: Find the total number of crimes recorded in the CRIME table.

result = %sql SELECT COUNT(*) FROM CRIME_DATA;

# Display the result
print(result)

 * sqlite:///FinalDB.db
Done.
+----------+
| COUNT(*) |
+----------+
|   533    |
+----------+


In [15]:
# Problem 2: List community area names and numbers with per capita income less than 11000

result = %sql SELECT community_area_name, community_area_number \
              FROM CENSUS_DATA \
              WHERE per_capita_income < 11000;

# Display the result
print(result)

 * sqlite:///FinalDB.db
Done.
+---------------------+-----------------------+
| COMMUNITY_AREA_NAME | COMMUNITY_AREA_NUMBER |
+---------------------+-----------------------+
|  West Garfield Park |          26.0         |
|    South Lawndale   |          30.0         |
|     Fuller Park     |          37.0         |
|      Riverdale      |          54.0         |
+---------------------+-----------------------+


In [16]:
# Problem 3: List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis

result = %sql SELECT case_number \
              FROM CRIME_DATA \
              WHERE description LIKE '%MINOR%';

# Display the result
print(result)

 * sqlite:///FinalDB.db
Done.
+-------------+
| CASE_NUMBER |
+-------------+
|   HL266884  |
|   HK238408  |
+-------------+


In [27]:
# Problem 4: Query to select all kidnapping crimes involving a child

# Adjust the query based on the actual column names
result = %sql SELECT * FROM CRIME_DATA WHERE primary_type = 'KIDNAPPING' AND description LIKE '%CHILD%';

# Display the result
print(result)



 * sqlite:///FinalDB.db
Done.
+---------+-------------+------------+----------------------+------+--------------+--------------------------+----------------------+--------+----------+------+----------+------+-----------------------+---------+--------------+--------------+------+-------------+--------------+-------------------------------+
|    ID   | CASE_NUMBER |    DATE    |        BLOCK         | IUCR | PRIMARY_TYPE |       DESCRIPTION        | LOCATION_DESCRIPTION | ARREST | DOMESTIC | BEAT | DISTRICT | WARD | COMMUNITY_AREA_NUMBER | FBICODE | X_COORDINATE | Y_COORDINATE | YEAR |   LATITUDE  |  LONGITUDE   |            LOCATION           |
+---------+-------------+------------+----------------------+------+--------------+--------------------------+----------------------+--------+----------+------+----------+------+-----------------------+---------+--------------+--------------+------+-------------+--------------+-------------------------------+
| 5276766 |   HN144152  | 2007-01-26 

In [30]:
# Problem 5: List the kind of crimes that were recorded at schools. (No repetitions)

result = %sql SELECT DISTINCT primary_type FROM CRIME_DATA WHERE location_description LIKE '%SCHOOL%';


# Display the result
print(result)


 * sqlite:///FinalDB.db
Done.
+------------------------+
|      PRIMARY_TYPE      |
+------------------------+
|        BATTERY         |
|    CRIMINAL DAMAGE     |
|       NARCOTICS        |
|        ASSAULT         |
|   CRIMINAL TRESPASS    |
| PUBLIC PEACE VIOLATION |
+------------------------+


In [33]:
# Query to list all columns in PUBLIC_SCHOOLS
result = %sql PRAGMA table_info(PUBLIC_SCHOOLS);

# Display the result
print(result)


 * sqlite:///FinalDB.db
Done.
+-----+--------------------------------------------------+---------+---------+------------+----+
| cid |                       name                       |   type  | notnull | dflt_value | pk |
+-----+--------------------------------------------------+---------+---------+------------+----+
|  0  |                    School_ID                     | INTEGER |    0    |    None    | 0  |
|  1  |                  NAME_OF_SCHOOL                  |   TEXT  |    0    |    None    | 0  |
|  2  |        Elementary, Middle, or High School        |   TEXT  |    0    |    None    | 0  |
|  3  |                  Street_Address                  |   TEXT  |    0    |    None    | 0  |
|  4  |                       City                       |   TEXT  |    0    |    None    | 0  |
|  5  |                      State                       |   TEXT  |    0    |    None    | 0  |
|  6  |                     ZIP_Code                     | INTEGER |    0    |    None    | 0  |


In [34]:
# Problem 6: List the type of schools along with the average safety score for each type

result = %sql SELECT "Elementary, Middle, or High School" AS school_type, AVG(SAFETY_SCORE) AS average_safety_score FROM PUBLIC_SCHOOLS GROUP BY "Elementary, Middle, or High School";

# Display the result
print(result)


 * sqlite:///FinalDB.db
Done.
+-------------+----------------------+
| school_type | average_safety_score |
+-------------+----------------------+
|      ES     |  49.52038369304557   |
|      HS     |  49.62352941176471   |
|      MS     |         48.0         |
+-------------+----------------------+


In [35]:
# Query to list all columns in CENSUS_DATA

result = %sql PRAGMA table_info(CENSUS_DATA);

# Display the result
print(result)

 * sqlite:///FinalDB.db
Done.
+-----+----------------------------------------------+---------+---------+------------+----+
| cid |                     name                     |   type  | notnull | dflt_value | pk |
+-----+----------------------------------------------+---------+---------+------------+----+
|  0  |            COMMUNITY_AREA_NUMBER             |   REAL  |    0    |    None    | 0  |
|  1  |             COMMUNITY_AREA_NAME              |   TEXT  |    0    |    None    | 0  |
|  2  |          PERCENT_OF_HOUSING_CROWDED          |   REAL  |    0    |    None    | 0  |
|  3  |       PERCENT_HOUSEHOLDS_BELOW_POVERTY       |   REAL  |    0    |    None    | 0  |
|  4  |         PERCENT_AGED_16__UNEMPLOYED          |   REAL  |    0    |    None    | 0  |
|  5  | PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA |   REAL  |    0    |    None    | 0  |
|  6  |       PERCENT_AGED_UNDER_18_OR_OVER_64       |   REAL  |    0    |    None    | 0  |
|  7  |              PER_CAPITA_INCOME  

In [36]:
# Problem 7: List 5 community areas with highest % of households below poverty line

result = %sql SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CENSUS_DATA ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5;

# Display the result
print(result)

 * sqlite:///FinalDB.db
Done.
+---------------------+----------------------------------+
| COMMUNITY_AREA_NAME | PERCENT_HOUSEHOLDS_BELOW_POVERTY |
+---------------------+----------------------------------+
|      Riverdale      |               56.5               |
|     Fuller Park     |               51.2               |
|      Englewood      |               46.6               |
|    North Lawndale   |               43.1               |
|  East Garfield Park |               42.4               |
+---------------------+----------------------------------+


In [37]:
# Problem 8: Which community area is most crime prone? Display the coumminty area number only

result = %sql SELECT COMMUNITY_AREA_NUMBER FROM CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(*) DESC LIMIT 1;

# Display the result
print(result)

 * sqlite:///FinalDB.db
Done.
+-----------------------+
| COMMUNITY_AREA_NUMBER |
+-----------------------+
|          25.0         |
+-----------------------+


In [38]:
# Problem 9: Use a sub-query to find the name of the community area with highest hardship index

result = %sql SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA);

# Display the result
print(result)

 * sqlite:///FinalDB.db
Done.
+---------------------+
| COMMUNITY_AREA_NAME |
+---------------------+
|      Riverdale      |
+---------------------+


In [39]:
# Problem 10: Use a sub-query to determine the Community Area Name with most number of crimes?

result = %sql SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE COMMUNITY_AREA_NUMBER = (SELECT COMMUNITY_AREA_NUMBER FROM (SELECT COMMUNITY_AREA_NUMBER, COUNT(*) AS crime_count FROM CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY crime_count DESC LIMIT 1));

# Display the result
print(result)

 * sqlite:///FinalDB.db
Done.
+---------------------+
| COMMUNITY_AREA_NAME |
+---------------------+
|        Austin       |
+---------------------+
